# 드라이브 연동

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/MyDrive/ML_Project

Mounted at /content/drive
/content/drive/MyDrive/ML_Project


# 모델 및 벡터 로드

In [29]:
import joblib
import pandas as pd

# Logistic Regression 최종 모델 로드
model = joblib.load("final_model.pkl")
tfidf = joblib.load("tfidf_vectorizer.pkl")

print("모델 및 벡터 로드 완료")

모델 및 벡터 로드 완료


# 전처리(기존과 동일)

In [45]:
import re

# Training 때 사용한 stopword 리스트 그대로 복붙
stopwords = set([
    # 1) filler / 말버릇
    "어", "음", "자", "뭐", "요", "네", "막", "그냥",
    "근데", "그죠", "거죠", "예",

    # 2) 지시어 (정보 없음)
    "이거", "이게", "이건", "그거", "그게",
    "저거", "요거", "요게", "얘는",

    # 3) 공손/형식적 표현
    "합니다", "되었습니다", "됩니다", "하겠습니다",
    "해주세요", "드릴", "보겠습니다", "할게요",
    "있습니다", "있어요", "있죠",

    # 4) 기능어 (기본 조사)
    "이", "그", "저", "을", "를", "은", "는", "에",
    "에서", "로", "것", "거", "건", "것들",
])

def preprocess(text):
    text = str(text).lower()                      # 소문자 변환
    text = re.sub(r"[^가-힣a-zA-Z0-9\s]", " ", text)  # 한글/영문/숫자 제외 모두 제거
    text = re.sub(r"\s+", " ", text).strip()      # 중복 공백 제거

    # 토큰화
    tokens = text.split()

    # Stopword 제거
    tokens = [t for t in tokens if t not in stopwords]

    # Training과 완전히 동일하게 문장 재구성
    return " ".join(tokens)

# 단일 문장 추론

In [46]:
def predict_sentence(sentence, threshold=0.06):
    clean = preprocess(sentence)
    vec = tfidf.transform([clean])
    prob_1 = model.predict_proba(vec)[0][1]
    pred = 1 if prob_1 >= threshold else 0
    return pred, prob_1

# 여러 문장 리스트 추론

In [47]:
def predict_batch(sentences, threshold=0.06):
    clean_list = [preprocess(s) for s in sentences]
    vec = tfidf.transform(clean_list)
    prob_1 = model.predict_proba(vec)[:, 1]
    preds = (prob_1 >= threshold).astype(int)
    return preds, prob_1

# 테스트

In [61]:
example_sentences = [
    "이번 알고리즘의 핵심 아이디어는 데이터를 반복적으로 갱신하는 것입니다.",
    "이 모델의 목적은 입력 문장에서 핵심 정보를 자동으로 추출하는 것입니다.",
    "여기까지 첫 번째 파트 설명이었습니다.",
    "혹시 질문 있으신가요?"
]

preds, probs = predict_batch(example_sentences, threshold=0.06)

for s, p, pr in zip(example_sentences, preds, probs):
    print(f"[문장] {s}")
    print(f"=> 예측 라벨: {p} (확률: {pr:.4f})")
    print("   • 1 = 핵심 문장 / 0 = 비핵심 문장\n")

[문장] 이번 알고리즘의 핵심 아이디어는 데이터를 반복적으로 갱신하는 것입니다.
=> 예측 라벨: 1 (확률: 0.1262)
   • 1 = 핵심 문장 / 0 = 비핵심 문장

[문장] 이 모델의 목적은 입력 문장에서 핵심 정보를 자동으로 추출하는 것입니다.
=> 예측 라벨: 1 (확률: 0.0617)
   • 1 = 핵심 문장 / 0 = 비핵심 문장

[문장] 여기까지 첫 번째 파트 설명이었습니다.
=> 예측 라벨: 0 (확률: 0.0329)
   • 1 = 핵심 문장 / 0 = 비핵심 문장

[문장] 혹시 질문 있으신가요?
=> 예측 라벨: 0 (확률: 0.0080)
   • 1 = 핵심 문장 / 0 = 비핵심 문장



In [65]:
while True:
    s = input("문장을 입력하세요 (종료: quit): ")
    if s.lower() == "quit":
        break
    pred, prob = predict_sentence(s)
    print(f"예측 라벨: {pred} (확률: {prob:.4f})  /  1=핵심, 0=비핵심\n")

문장을 입력하세요 (종료: quit): 분할 정복(Divide and Conquer) 알고리즘은 하나의 큰 문제를 작은 하위 문제로 분할(Divide)하고, 각 하위 문제를 해결(Conquer)한 뒤, 그 해결책들을 합쳐서(Combine) 원래의 문제를 해결하는 알고리즘 설계 기법입니다.
예측 라벨: 1 (확률: 0.2749)  /  1=핵심, 0=비핵심

문장을 입력하세요 (종료: quit): 여러분들 전부 이해하셨죠?
예측 라벨: 0 (확률: 0.0502)  /  1=핵심, 0=비핵심

문장을 입력하세요 (종료: quit): 아 네 그러면 여기까지 하겠습니다.
예측 라벨: 0 (확률: 0.0254)  /  1=핵심, 0=비핵심

문장을 입력하세요 (종료: quit): 잠시만요, 화면이 안 보이네요.
예측 라벨: 0 (확률: 0.0081)  /  1=핵심, 0=비핵심

문장을 입력하세요 (종료: quit): quit
